# 🌟 Personalized Week Planner Demo
This notebook demonstrates how to build a week plan using weather, events, and recipes based on location and dates.

Technologies used:
- LangGraph
- LangChain Tools
- External APIs: WeatherAPI, Eventbrite


In [4]:
#Import Dependencies 

import requests
import dateparser
from datetime import datetime, timedelta
import json
from typing import List, Dict, Any, Annotated, Literal
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from typing_extensions import TypedDict

In [5]:
# Load API Keys
key = json.load(open("keys.json"))
WEATHER_API_KEY = key["weather_api"]
EVENTBRITE_TOKEN = key["eventbrite_api"]

In [6]:
# Helper Functions Section

def resolve_dates(dates: List[str]) -> List[datetime.date]:
    resolved = []
    for d in dates:
        parsed = dateparser.parse(d, settings={'PREFER_DATES_FROM': 'future'})
        if parsed:
            resolved.append(parsed.date())
    return resolved

def fetch_weather(location: str, target_date: datetime.date) -> Dict[str, Any]:
    days_diff = (target_date - datetime.now().date()).days
    if days_diff < 0 or days_diff > 14:
        return {"error": "Date out of range (must be within 14 days)"}

    url = f"http://api.weatherapi.com/v1/forecast.json"
    response = requests.get(url, params={
        "key": WEATHER_API_KEY,
        "q": location,
        "days": days_diff + 1,
        "aqi": "no"
    })
    data = response.json()

    for day in data["forecast"]["forecastday"]:
        if day["date"] == target_date.strftime("%Y-%m-%d"):
            return {
                "condition": day["day"]["condition"]["text"],
                "max_temp_c": day["day"]["maxtemp_c"],
                "min_temp_c": day["day"]["mintemp_c"],
                "rain_chance": day["day"]["daily_chance_of_rain"],
                "wind_kph": day["day"]["maxwind_kph"],
                "is_outdoor_friendly": day["day"]["daily_chance_of_rain"] < 30
            }
    return {"error": "Weather data not found for target date"}

def fetch_event(location: str, date: datetime.date, outdoor: bool) -> Dict[str, str]:
    date_iso = date.isoformat() + "T00:00:00Z"
    url = f"https://www.eventbriteapi.com/v3/events/search/"
    headers = {"Authorization": f"Bearer {EVENTBRITE_TOKEN}"}
    params = {
        "location.address": location,
        "start_date.range_start": date_iso,
        "sort_by": "date",
        "page_size": 1
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        return {"event": "No event found or API error."}

    events = response.json().get("events", [])
    if not events:
        return {"event": "No event found."}

    event = events[0]
    return {
        "name": event.get("name", {}).get("text", "Unnamed Event"),
        "url": event.get("url", "#")
    }

In [8]:
#Plan Generation Function

def generate_day_plan(location: str, date: datetime.date) -> str:
    weather = fetch_weather(location, date)
    if "error" in weather:
        return f"\n📅 {date.strftime('%A, %B %d')}: Unable to fetch weather data."

    event = fetch_event(location, date, weather["is_outdoor_friendly"])

    return (
        f"\n📅 {date.strftime('%A, %B %d')}:\n"
        f"- 🌤️ Weather: {weather['condition']}, {weather['max_temp_c']}°C / {weather['min_temp_c']}°C\n"
        f"- 🎟️ Event: {event.get('name')} → [View Event]({event.get('url')})\n"
    )

def generate_week_plan(location: str, days: List[str]) -> str:
    dates = resolve_dates(days)
    if not dates:
        return "Couldn't resolve any dates from your input."
    plans = [generate_day_plan(location, d) for d in dates]
    return "\n".join(plans)

In [9]:
# Define the week planner tool

@tool
def plan_week(location: str, days: str = "") -> str:
    """
    Generates a personalized activity and food plan based on weather, events, and recipe suggestions.

    Args:
        location: City or location name.
        days: Comma-separated days or dates 
              If blank, defaults to planning for the next 7 days.

    Returns:
        A multi-day plan with weather, event, and food suggestions.
    """
    if not days.strip():
        today = datetime.now().date()
        day_list = [(today + timedelta(days=i)).strftime("%A") for i in range(7)]
    else:
        day_list = [d.strip() for d in days.split(",")]

    return generate_week_plan(location, day_list)

In [10]:
# Setup LangGraph

llm = ChatOpenAI(
    api_key=key["openai_api"],
    model="gpt-4o",
)

tool_node = ToolNode([plan_week])

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph = StateGraph(State)
graph.add_node("tool_node", tool_node)

def prompt_node(state: State) -> State:
    new_message = llm_with_tools.invoke(state["messages"])
    return {"messages": [new_message]}

graph.add_node("prompt_node", prompt_node)

def conditional_edge(state: State) -> Literal["tool_node", "__end__"]:
    last_message = state["messages"][-1]
    return "tool_node" if last_message.tool_calls else "__end__"

graph.add_conditional_edges("prompt_node", conditional_edge)
graph.add_edge("tool_node", "prompt_node")


graph.set_entry_point("prompt_node")


llm_with_tools = llm.bind_tools([plan_week])
APP = graph.compile()

In [11]:
# Try a sample user query
user_query = "Plan my weekend in Chicago. I like staying indoor with occasional strolling."

response = APP.invoke({"messages": [user_query]})
print(response["messages"][-1].content)

Here's a personalized plan for your weekend in Chicago, perfect for indoor relaxation with occasional outdoor strolling:

### Saturday, April 26:
- **Weather:** Overcast, with temperatures around 6.8°C (high) / 4.1°C (low).
- **Activity:** 
  - Enjoy a cozy day at home. How about starting a new book or diving into a movie marathon with your favorite snacks? 
  - If you feel like stepping out, a short stroll in a neighborhood park with a hot drink can be refreshing.

### Sunday, April 27:
- **Weather:** Misty, with temperatures around 16.0°C (high) / 3.3°C (low).
- **Activity:** 
  - Perfect day for visiting a cafe or a local museum in the morning when it's misty, enjoying the quiet indoors.
  - In the afternoon, as it warms up, consider a walk down the Chicago Riverwalk or through Millennium Park. 

No major public events are happening over the weekend, so you can enjoy a peaceful, self-paced time. Make sure to stay comfy and have fun with your chosen indoor activities!
